In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings 
from langchain.document_loaders import UnstructuredFileLoader
from langchain.document_loaders import DirectoryLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import RecursiveCharacterTextSplitter


embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-mpnet-base-dot-v1")

#with open(r"C:\Users\Francesco\Desktop\The Psychology of Totalitarianism 2022- Mattias Desmet.txt", encoding='utf-8') as f:
#    book = f.read()

#loader = [UnstructuredFileLoader(r"C:\Users\Francesco\Desktop\The Psychology of Totalitarianism 2022- Mattias Desmet.pdf", encoding='utf-8'), UnstructuredFileLoader(r"C:\Users\Francesco\Desktop\The unhappiness machine.pdf", encoding='utf-8')]
#raw_documents = AnalyticDB.from_documents(loader, )

loader = DirectoryLoader(r"C:\Users\Francesco\Documents\GITHUB\Self-Hosted-QA\books", silent_errors=True, show_progress=True)
raw_documents = loader.load()

# Split text
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap  = 20)
documents = text_splitter.split_documents(raw_documents)

docsearch = Chroma.from_documents(
    documents, embeddings, persist_directory="db", metadatas=[{"source": f"{i}-pl"} for i in range(len(documents))]
)
docsearch.persist()
docsearch = None

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings 
from langchain.document_loaders import UnstructuredFileLoader
from langchain.document_loaders import DirectoryLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import RecursiveCharacterTextSplitter


embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-mpnet-base-dot-v1")
docsearch = Chroma(persist_directory="db", embedding_function=embeddings)

In [ ]:
## query the database using similarity search


query = "What happened with the inventions of the hard disk?"

if query == "exit":
    pass
else:
    docs = docsearch.similarity_search_with_score(query, k=3)
    for doc in docs:
        print(doc[0].page_content)
        print(doc[0].metadata["source"])

In [7]:
from langchain.chains import RetrievalQA
from langchain import HuggingFacePipeline
from langchain.vectorstores import Chroma

llm = HuggingFacePipeline.from_model_id(model_id="bigscience/bloom-560m", task="text-generation", model_kwargs={"temperature":0, "max_length":1000})
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(), return_source_documents=True)

In [ ]:
qa({"query": "What happened with the inventions of the hard disk?"}, return_only_outputs=True)